In [ ]:
!pip install zeyrek

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import re
import string
import zeyrek
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = pd.read_csv("all_tweets.csv")

In [ ]:
#Belirlenmiş olan kategoriler tanımlanıyor

category_0 = "spor"
category_1 = "ekonomi"
category_2 = "siyaset"
category_3 = "teknoloji & bilim"


#Datasetteki tweetler kategorilere göre ayrılıyor

tweets_0 = df[df.Category == category_0] #spor
tweets_1 = df[df.Category == category_1] #ekonomi
tweets_2 = df[df.Category == category_2] #siyaset
tweets_3 = df[df.Category == category_3] #teknoloji & bilim

#Datasetinde sadece tweetlerin bulunduğu kolon alınıyor

tweets_0 = tweets_0.Tweet.to_numpy()
tweets_1 = tweets_1.Tweet.to_numpy()
tweets_2 = tweets_2.Tweet.to_numpy()
tweets_3 = tweets_3.Tweet.to_numpy()

tweets = [tweets_0, tweets_1, tweets_2, tweets_3]
tweets_category = [category_0, category_1, category_2, category_3]



In [ ]:
analyzer = zeyrek.MorphAnalyzer() 
stop_words = set(stopwords.words('turkish')) #stopwordslerin temizlenmesi için stopwordsleri alıyoruz

In [ ]:
#twitterdan çekilen verilerin içerisindeki emojileri silmek için gerekli olan emoji kodları

emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)

In [ ]:
tweet=[]
labels=[]
nsw=[]

In [ ]:
#tweetlerin içerisindeki breakline, noktalama işaretleri, emojiler,stopwordsler gibi istenmeyen veriler çıkarılıyor ve lemmatize işlemi yapılıyor.

for i, tweets_set in enumerate(tweets):
    for tweet_string in tqdm(tweets_set):  

          tweet_string = tweet_string.replace('\n','') #Line breaks lerden temizliyoruz
          tweet_string = re.sub('http://\S+|https://\S+', '', tweet_string) #tweetler içerisindeki linkler temizleniyor
          tweet_string = tweet_string.translate(str.maketrans("", "", string.punctuation)) #tweetlerdeki noktalama işaretleri temizleniyor
          tweet_string = emoji_pattern.sub(r'', tweet_string)  #tweet içerisindeki emojiler temizleniyor
          lemm_tweet = analyzer.lemmatize(tweet_string)  #tweetler için lemmatize işlemi gerçekleştiriliyor
          lemm_tweet=[el[1][-1] for el in lemm_tweet]
          for r in lemm_tweet:     
              if not r in stop_words: 
                  nsw.append(r)       #stopwords olmayanlar ayrılıyor ve aşağıda birleştiriliyor
                  
          clear_tweet =' '.join(nsw)  #dizi string hale getiriliyor
          nsw=[]
          clear_tweet = clear_tweet.lower() #bütün tweetler küçük harflere çevriliyor
          tweet.append(clear_tweet) #temizlenmiş tweetin kendisi listeye ekleniyor
     

In [ ]:
df["Clean"]=tweet                 #temizlenmiş veriler dataframeye clean kolonu altına kaydediliyor
df.to_csv('clean_all_tweets.csv') #dataframe csv dosyası olarak kaydediliyor